In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [45]:
env.reset()
print(env.P[1][0])

[(1.0, 1, -1.0, False)]


In [58]:
def print_result(v, rsize):
    print('| ')
    for i in range(len(v)):
        if i % rsize == 0:
            print('\n', end='| ')
        print(v[i], end='| ')

In [87]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    print_result(V, env.nA)
    while True:
        env.reset()
        delta = 0
        for s in range(env.nS):
            vs = 0
            for a in range(env.nA):
                for prob, next_s, r, done in env.P[s][a]:
#                     if not done:
                    vs += policy[s][a] * prob * (r + discount_factor *  V[next_s])
            delta = max(delta, np.abs(vs - V[s]))
            V[s] = vs
        if delta <= theta:
            break
    return np.array(V)

In [88]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)
print(v)

| 

| 0.0| 0.0| 0.0| 0.0| 
| 0.0| 0.0| 0.0| 0.0| 
| 0.0| 0.0| 0.0| 0.0| 
| 0.0| 0.0| 0.0| 0.0| [  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569
   0.        ]


In [86]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)